## Imports

In [1]:
import sys
import random
from pathlib import Path
from collections import Counter

import datasets

from utils import combine_strings_with_whitespace

In [2]:
ELK_PATH = Path("../../../elk/")
ELK_PATH.resolve()

PosixPath('/fsx/home-augustas/elk')

In [3]:
modules = [
    ELK_PATH,
    ELK_PATH / "elk" / "promptsource",
]

for module in modules:
    if not str(module) in sys.path:
        sys.path.insert(0, str(module.resolve()))

sys.path[:2]

['/fsx/home-augustas/elk/elk/promptsource', '/fsx/home-augustas/elk']

In [4]:
from templates import DatasetTemplates

In [5]:
# Disable the logging of the datasets library
import datasets

datasets.logging.set_verbosity_error()

# Load the dataset

In [6]:
VERSION = "v4"

In [7]:
!ls -lah datasets | grep {VERSION}

drwxr-xr-x  9 augustas Domain Users  25K Jun 29 11:14 burns_datasets_VINC_ppo_training_raw_v4
drwxr-xr-x  9 augustas Domain Users  33K Jun 29 11:14 burns_datasets_VINC_raw_v4
-rw-r--r--  1 augustas Domain Users 2.8M Jun 29 10:48 burns_datasets_VINC_train_v4.parquet
-rw-r--r--  1 augustas Domain Users 380K Jun 29 11:18 burns_datasets_VINC_validation_v4.parquet


In [8]:
# Set dataset_dict to test_dataset
dataset_dict = datasets.DatasetDict.load_from_disk(
    f"datasets/burns_datasets_VINC_ppo_training_raw_{VERSION}"
)
dataset_dict

DatasetDict({
    ag_news: Dataset({
        features: ['text', 'label'],
        num_rows: 1380
    })
    amazon_polarity: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 1380
    })
    dbpedia_14: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 1380
    })
    glue/qnli: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 1380
    })
    imdb: Dataset({
        features: ['text', 'label'],
        num_rows: 1380
    })
    piqa: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 1380
    })
    super_glue/boolq: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 1380
    })
    super_glue/copa: Dataset({
        features: ['premise', 'choice1', 'choice2', 'question', 'idx', 'label'],
        num_rows: 89
    })
    super_glue/rte: Dataset({
        features: ['premise', 'hypothesis', 'idx', 'label'],
        num_rows: 246
  

In [9]:
sum(dataset.num_rows for dataset in dataset_dict.values())

9995

In [10]:
for dataset_name, dataset in dataset_dict.items():
    print(f"{dataset_name}: {len(Counter(dataset['label']))}")

ag_news: 4
amazon_polarity: 2
dbpedia_14: 14
glue/qnli: 2
imdb: 2
piqa: 2
super_glue/boolq: 2
super_glue/copa: 2
super_glue/rte: 2


## Load the templates

In [11]:
dataset_template_dict = {}

for dataset_path in dataset_dict.keys():
    dataset_templates = DatasetTemplates(dataset_path)

    dataset_templates.templates = {
        x.name: x for x in dataset_templates.templates.values() if x.get_answer_choices_list(dataset_dict[dataset_path][0]) is not None
    }

    dataset_template_dict[dataset_path] = dataset_templates

In [12]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

ag_news: 15
amazon_polarity: 11
dbpedia_14: 11
glue/qnli: 5
imdb: 13
piqa: 7
super_glue/boolq: 10
super_glue/copa: 9
super_glue/rte: 11


In [13]:
# dataset_template_dict["ag_news"].templates

In [14]:
for dataset_name in dataset_template_dict:
    good_templates = {
        name: x for name, x in dataset_template_dict[dataset_name].templates.items() if x.metadata.choices_in_prompt
    }
    dataset_template_dict[dataset_name].templates = good_templates
    print(f"{dataset_name}: {len(good_templates)}")

ag_news: 12
amazon_polarity: 8
dbpedia_14: 11
glue/qnli: 1
imdb: 4
piqa: 6
super_glue/boolq: 5
super_glue/copa: 9
super_glue/rte: 11


In [15]:
for dataset_name, dataset_templates in dataset_template_dict.items():
    print(f"{dataset_name}: {len(dataset_templates.templates)}")

ag_news: 12
amazon_polarity: 8
dbpedia_14: 11
glue/qnli: 1
imdb: 4
piqa: 6
super_glue/boolq: 5
super_glue/copa: 9
super_glue/rte: 11


In [16]:
%%time

# Reproducibility
random.seed(2023)

ALLOWED_KEYS = ["prompt", "best_response", "original_dataset", "template_name"]

new_dataset = []

for dataset_name, dataset in dataset_dict.items():
    print(dataset_name)

    for idx, entry in enumerate(dataset):
        new_entry = entry.copy()
        
        # In case we need to know which dataset the entry came from
        new_entry["original_dataset"] = dataset_name

        # Sample a random template
        template_name = random.choice(
            list(dataset_template_dict[dataset_name].templates.keys())
        )
        template = dataset_template_dict[dataset_name].templates[template_name]
        new_entry["template_name"] = template_name

        q, a = template.apply(new_entry)
        new_entry["prompt"] = combine_strings_with_whitespace(a, q)
        if not new_entry["prompt"][-1].isspace():
            new_entry["prompt"] += " "

        # We can now change the label to whether the sample is truthful or not
        new_entry["best_response"] = a.strip()

        # Remove all other keys
        new_entry = { k: v for k, v in new_entry.items() if k in ALLOWED_KEYS }

        # Append to the new dataset
        new_dataset.append(new_entry)

ag_news


amazon_polarity
dbpedia_14
glue/qnli
imdb
piqa
super_glue/boolq
super_glue/copa
super_glue/rte
CPU times: user 19.3 s, sys: 115 ms, total: 19.4 s
Wall time: 19.5 s


In [17]:
my_dataset = datasets.Dataset.from_list(new_dataset)
my_dataset

Dataset({
    features: ['original_dataset', 'template_name', 'prompt', 'best_response'],
    num_rows: 9995
})

In [22]:
current_idx = 0
my_dataset[current_idx]

{'original_dataset': 'ag_news',
 'template_name': 'burns_4',
 'prompt': 'Fujitsu Siemens profit grows 60 percent Fujitsu Siemens Computers (Holding) BV, Europe #39;s largest remaining computer manufacturer, posted a 60 percent leap in profit for the first half of its fiscal year on higher sales of laptops and servers to business customers, the company said Wednesday. Choice 1: Science and technology. Choice 2: Business. Between choice 1 and choice 2, the topic of this example is ',
 'best_response': 'Science and technology'}

In [24]:
print(f'\'{my_dataset[current_idx]["prompt"]}\'')

'Fujitsu Siemens profit grows 60 percent Fujitsu Siemens Computers (Holding) BV, Europe #39;s largest remaining computer manufacturer, posted a 60 percent leap in profit for the first half of its fiscal year on higher sales of laptops and servers to business customers, the company said Wednesday. Choice 1: Science and technology. Choice 2: Business. Between choice 1 and choice 2, the topic of this example is '


In [25]:
Counter(my_dataset["original_dataset"])

Counter({'ag_news': 1380,
         'amazon_polarity': 1380,
         'dbpedia_14': 1380,
         'glue/qnli': 1380,
         'imdb': 1380,
         'piqa': 1380,
         'super_glue/boolq': 1380,
         'super_glue/rte': 246,
         'super_glue/copa': 89})

In [28]:
# my_dataset.to_parquet(f"datasets/burns_datasets_VINC_ppo_training_{VERSION}.parquet")

In [27]:
!ls -lah datasets | grep {VERSION}

drwxr-xr-x  9 augustas Domain Users  25K Jun 29 11:14 burns_datasets_VINC_ppo_training_raw_v4
-rw-r--r--  1 augustas Domain Users 3.0M Jun 29 11:21 burns_datasets_VINC_ppo_training_v4.parquet
drwxr-xr-x  9 augustas Domain Users  33K Jun 29 11:14 burns_datasets_VINC_raw_v4
-rw-r--r--  1 augustas Domain Users 2.8M Jun 29 10:48 burns_datasets_VINC_train_v4.parquet
-rw-r--r--  1 augustas Domain Users 380K Jun 29 11:18 burns_datasets_VINC_validation_v4.parquet
